In [1]:
import numpy as np
from sklearn.pipeline import Pipeline

import GEOparse
from Genobolitics import *
from sklearn_utils.preprocessing import FoldChangeScaler
from metabolitics.preprocessing import MetaboliticsTransformer, MetaboliticsPipeline

import pickle

In [2]:
breast_cancer_dataset = GEOparse.get_GEO('GDS5819').table

04-Jan-2018 23:50:32 INFO GEOparse - File already exist: using local version.
04-Jan-2018 23:50:32 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
04-Jan-2018 23:50:32 DEBUG GEOparse - DATABASE: Geo
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_1
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_2
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_3
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_4
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [3]:
unhealthy_raw='\
#GSM1599177 = Value for GSM1599177: Primary human monocytes from patient with metastatic breast cancer #1; src: Primary human monocytes\
#GSM1599178 = Value for GSM1599178: Primary human monocytes from patient with metastatic breast cancer #2; src: Primary human monocytes\
#GSM1599179 = Value for GSM1599179: Primary human monocytes from patient with metastatic breast cancer #3; src: Primary human monocytes\
#GSM1599180 = Value for GSM1599180: Primary human monocytes from patient with metastatic breast cancer #4; src: Primary human monocytes'

In [4]:
healthy_raw='\
#GSM1599181 = Value for GSM1599181: Primary human monocytes from healthy control #1; src: Primary human monocytes\
#GSM1599182 = Value for GSM1599182: Primary human monocytes from healthy control #2; src: Primary human monocytes\
#GSM1599183 = Value for GSM1599183: Primary human monocytes from healthy control #3; src: Primary human monocytes'

In [5]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [6]:
print(len(healthy), len(unhealthy))

3 4


In [7]:
healthy & unhealthy

set()

In [8]:
breast_cancer_dataset[breast_cancer_dataset.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM1599177,GSM1599178,GSM1599179
0,ILMN_1343048,ILMN_1343048,NaN,NaN,NaN
1,ILMN_1343049,ILMN_1343049,NaN,NaN,NaN
2,ILMN_1343050,ILMN_1343050,NaN,NaN,NaN
3,ILMN_1343052,ILMN_1343052,NaN,NaN,NaN
4,ILMN_1343059,ILMN_1343059,NaN,NaN,NaN


In [9]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [10]:
X, y = parse_database('GDS5819', labels=dict(labels_))

04-Jan-2018 23:50:32 INFO GEOparse - File already exist: using local version.
04-Jan-2018 23:50:32 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
04-Jan-2018 23:50:32 DEBUG GEOparse - DATABASE: Geo
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_1
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_2
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_3
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_4
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


GSM1599180 added with length 9745
GSM1599182 added with length 9745
GSM1599177 added with length 9745
GSM1599179 added with length 9745
GSM1599178 added with length 9745
GSM1599183 added with length 9745
GSM1599181 added with length 9745


In [11]:
pickle.dump(X, open("datasets/breast_cancer_X","wb"))
pickle.dump(y, open("datasets/breast_cancer_y","wb"))

In [ ]:
X = pickle.load(open("datasets/breast_cancer_X","rb"))
y = pickle.load(open("datasets/breast_cancer_y","rb"))

In [ ]:
genobolitics_transformer = MetaboliticsTransformer()
genobolitics_transformer.analyzer = Genobolitics("recon2")

pipeline = Pipeline([('scaling', FoldChangeScaler(reference_label='healthy')),
                     ('LP_FVA', genobolitics_transformer)])

results = pipeline.fit_transform(X, y)

pickle.dump(results, open("breast_cancer.results", "wb"))

could not calculate fold changes for 13DAMPPOX: 13dampp_c + h2o_c + o2_c --> bamppald_c + h2o2_c + nh4_c
could not calculate fold changes for 24_25VITD2Hm: 25hvitd2_m + h_m + nadph_m + o2_m --> 2425dhvitd2_m + h2o_m + nadp_m
could not calculate fold changes for 24_25VITD3Hm: 25hvitd3_m + h_m + nadph_m + o2_m --> 2425dhvitd3_m + h2o_m + nadp_m
could not calculate fold changes for 25VITD2Hm: 25hvitd2_m + h_m + nadph_m + o2_m --> 1a25dhvitd2_m + h2o_m + nadp_m
could not calculate fold changes for 25VITD3Hm: 25hvitd3_m + h_m + nadph_m + o2_m --> 1a25dhvitd3_m + h2o_m + nadp_m
could not calculate fold changes for 2AMACHYD: 2amac_c + h2o_c --> nh4_c + pyr_c
could not calculate fold changes for 2HBO: 2hb_c + nad_c <=> 2obut_c + h_c + nadh_c
could not calculate fold changes for 34DHOXPEGOX: 34dhmald_c + h_c + nadh_c <=> 34dhoxpeg_c + nad_c
could not calculate fold changes for 34DHPLACOX: 34dhpac_c + h2o_c + nad_c --> 34dhpha_c + 2.0 h_c + nadh_c
could not calculate fold changes for 34DHPLACOX_

could not calculate fold changes for 41R2A1H12BOOX: h2o_c + nrpphr_c + o2_c --> 34dhmald_c + h2o2_c + nh4_c
could not calculate fold changes for 42A12BOOX: dopa_c + h2o_c + o2_c --> 34dhpac_c + h2o2_c + nh4_c
could not calculate fold changes for ACNAMPH: acnamp_c + h2o_c --> acnam_c + pi_c
could not calculate fold changes for 4HOXPACDOX_NADP_: 4hoxpacd_c + h2o_c + nadp_c --> 4hphac_c + 2.0 h_c + nadph_c
could not calculate fold changes for ACOAD10m: 2mbcoa_m + fad_m --> 2mb2coa_m + fadh2_m
could not calculate fold changes for 5ADTSTSTERONESULT: 5adtststerone_c + paps_c --> 5adtststerones_c + h_c + pap_c
could not calculate fold changes for ACOAD1fm: btcoa_m + fad_m --> b2coa_m + fadh2_m
could not calculate fold changes for 5HLTDL: 5htrp_c + h_c --> co2_c + srtn_c
could not calculate fold changes for ACOAD8m: fad_m + ivcoa_m --> 3mb2coa_m + fadh2_m
could not calculate fold changes for 5HOXINDACTO2OX: 5hoxindact_c + h2o_c + o2_c --> 5hoxindoa_c + h2o2_c + h_c
could not calculate fold cha

could not calculate fold changes for ACOAD8m: fad_m + ivcoa_m --> 3mb2coa_m + fadh2_m
could not calculate fold changes for 3SALACBOXL: 3sala_c + 2.0 h_c --> co2_c + hyptaur_c
could not calculate fold changes for AG13T17g: ksii_core4_pre4_g + uacgam_g --> h_g + ksii_core4_pre5_g + udp_g
could not calculate fold changes for AG13T18g: ksii_core4_pre7_g + uacgam_g --> h_g + ksii_core4_pre8_g + udp_g
could not calculate fold changes for ACOAHi: accoa_c + h2o_c --> ac_c + coa_c + h_c
could not calculate fold changes for AG13T1g: ksii_core2_pre2_g + uacgam_g --> h_g + ksii_core2_pre3_g + udp_g
could not calculate fold changes for AG13T2g: ksii_core2_pre4_g + uacgam_g --> h_g + ksii_core2_pre5_g + udp_g
could not calculate fold changes for ACOATA: ACP_c + accoa_c <=> acACP_c + coa_c
could not calculate fold changes for AG13T3g: ksii_core2_pre7_g + uacgam_g --> h_g + ksii_core2_pre8_g + udp_g
could not calculate fold changes for ACODA: acorn_c + h2o_c --> ac_c + orn_c
could not calculate fold c

could not calculate fold changes for ALDD20x: h2o_c + id3acald_c + nad_c --> 2.0 h_c + ind3ac_c + nadh_c
could not calculate fold changes for ALDD20xm: h2o_m + id3acald_m + nad_m --> 2.0 h_m + ind3ac_m + nadh_m
could not calculate fold changes for 3HKYNAKGAT: akg_c + hLkynr_c --> 42A3HP24DB_c + glu_L_c
could not calculate fold changes for AG13T10g: ksi_pre18_g + uacgam_g --> h_g + ksi_pre19_g + udp_g
could not calculate fold changes for 3HLYTCL: 34dhphe_c + h_c --> co2_c + dopa_c
could not calculate fold changes for AG13T11g: ksi_pre21_g + uacgam_g --> h_g + ksi_pre22_g + udp_g
could not calculate fold changes for AG13T12g: ksi_pre24_g + uacgam_g --> h_g + ksi_pre25_g + udp_g
could not calculate fold changes for ALDD2x: acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
could not calculate fold changes for AG13T13g: ksi_pre27_g + uacgam_g --> h_g + ksi_pre28_g + udp_g
could not calculate fold changes for 3HXKYNDCL: hLkynr_c + h_c --> 3hxkynam_c + co2_c
could not calculate fold changes

could not calculate fold changes for ABO1g: fucgalacglcgalgluside_hs_g + udpacgal_g --> acgalfucgalacglcgalgluside_hs_g + h_g + udp_g
could not calculate fold changes for 13DAMPPOX: 13dampp_c + h2o_c + o2_c --> bamppald_c + h2o2_c + nh4_c
could not calculate fold changes for ARGN: arg_L_c + h2o_c --> orn_c + urea_c
could not calculate fold changes for ABO2g: fucgalacglcgalgluside_hs_g + udpgal_g --> galfucgalacglcgalgluside_hs_g + h_g + udp_g
could not calculate fold changes for ARGNm: arg_L_m + h2o_m --> orn_m + urea_m
could not calculate fold changes for ACSOMT: 5hoxindoa_c + amet_c --> 5moxact_c + ahcys_c
could not calculate fold changes for ALASm: gly_m + h_m + succoa_m --> 5aop_m + co2_m + coa_m
could not calculate fold changes for ALATA_L: akg_c + ala_L_c <=> glu_L_c + pyr_c
could not calculate fold changes for ABO3g: fuc12gal14acglcgalgluside_hs_g + udpgal_g --> galfuc12gal14acglcgalgluside_hs_g + h_g + udp_g
could not calculate fold changes for ACSRTNMT: Nacsertn_c + amet_c -->

...

could not calculate fold changes for S6T14g: ksi_pre34_g + paps_g --> h_g + ksi_pre35_g + pap_g
could not calculate fold changes for S6T12g: ksi_pre28_g + paps_g --> h_g + ksi_pre29_g + pap_g
could not calculate fold changes for S6T16g: ksii_core4_pre5_g + paps_g --> h_g + ksii_core4_pre6_g + pap_g
could not calculate fold changes for S6T4g: ksi_pre4_g + paps_g --> h_g + ksi_pre5_g + pap_g
could not calculate fold changes for S6T9g: ksi_pre19_g + paps_g --> h_g + ksi_pre20_g + pap_g
could not calculate fold changes for S6T17g: ksii_core4_pre8_g + paps_g --> h_g + ksii_core4_pre9_g + pap_g
could not calculate fold changes for S6T13g: ksi_pre31_g + paps_g --> h_g + ksi_pre32_g + pap_g
could not calculate fold changes for S6T5g: ksi_pre7_g + paps_g --> h_g + ksi_pre8_g + pap_g
could not calculate fold changes for S6T16g: ksii_core4_pre5_g + paps_g --> h_g + ksii_core4_pre6_g + pap_g
could not calculate fold changes for S6T14g: ksi_pre34_g + paps_g --> h_g + ksi_pre35_g + pap_g
could not c

could not calculate fold changes for SELCYSTGL: h2o_c + selcyst_c --> 2obut_c + nh4_c + selcys_c
could not calculate fold changes for SACCD3m: akg_m + h_m + lys_L_m + nadph_m --> h2o_m + nadp_m + saccrp_L_m
could not calculate fold changes for SARDHm: fad_m + sarcs_m + thf_m --> fadh2_m + gly_m + mlthf_m
could not calculate fold changes for SELCYSTS: selhcys_c + ser_L_c --> h2o_c + selcyst_c
could not calculate fold changes for SACCD3m: akg_m + h_m + lys_L_m + nadph_m --> h2o_m + nadp_m + saccrp_L_m
could not calculate fold changes for SELCYSTS: selhcys_c + ser_L_c --> h2o_c + selcyst_c
could not calculate fold changes for SACCD4m: h2o_m + nadp_m + saccrp_L_m --> L2aadp6sa_m + glu_L_m + h_m + nadph_m
could not calculate fold changes for SBPP1er: h2o_r + sph1p_r --> pi_r + sphgn_r
could not calculate fold changes for SACCD4m: h2o_m + nadp_m + saccrp_L_m --> L2aadp6sa_m + glu_L_m + h_m + nadph_m
could not calculate fold changes for SELMETAT: atp_c + h2o_c + selmeth_c --> pi_c + ppi_c + s

could not calculate fold changes for SIAASE: 2.0 h2o_c + s2l2n2m2mn_c --> 2.0 acnam_c + l2n2m2mn_c
could not calculate fold changes for SPODMe: 2.0 h_e + 2.0 o2s_e --> h2o2_e + o2_e
could not calculate fold changes for SIAASE: 2.0 h2o_c + s2l2n2m2mn_c --> 2.0 acnam_c + l2n2m2mn_c
could not calculate fold changes for SMPD3g: h2o_g + sphmyln_hs_g --> cholp_g + crm_hs_g + 2.0 h_g
could not calculate fold changes for SPHK21c: atp_c + sphings_c --> adp_c + h_c + sphs1p_c
could not calculate fold changes for SPODMe: 2.0 h_e + 2.0 o2s_e --> h2o2_e + o2_e
could not calculate fold changes for SMPD3l: h2o_l + sphmyln_hs_l --> cholp_l + crm_hs_l + 2.0 h_l
could not calculate fold changes for SPR: 6pthp_c + 2.0 h_c + 2.0 nadph_c --> 2.0 nadp_c + thbpt_c
could not calculate fold changes for SLCBK1: atp_c + sphgn_c --> adp_c + h_c + sph1p_c
could not calculate fold changes for SPR: 6pthp_c + 2.0 h_c + 2.0 nadph_c --> 2.0 nadp_c + thbpt_c
could not calculate fold changes for SLDx: nad_c + sl_L_c <=> 

could not calculate fold changes for SRTNACT: accoa_c + srtn_c --> Nacsertn_c + coa_c + h_c
could not calculate fold changes for SRTNMTX: amet_c + srtn_c --> ahcys_c + h_c + nmthsrtn_c
could not calculate fold changes for ST8SIA52g: cmpacna_g + gt1b_hs_g --> cmp_g + gq1b_hs_g + 4.0 h_g
could not calculate fold changes for ST8SIA52g: cmpacna_g + gt1b_hs_g --> cmp_g + gq1b_hs_g + 4.0 h_g
could not calculate fold changes for SRTNMTX: amet_c + srtn_c --> ahcys_c + h_c + nmthsrtn_c
could not calculate fold changes for ST6GALNAC31: cmpacna_g + gm1b_hs_g --> cmp_g + gd1a_hs_g
could not calculate fold changes for ST8SIA53g: cmpacna_g + gd1a_hs_g --> cmp_g + gt1a_hs_g
could not calculate fold changes for ST8SIA53g: cmpacna_g + gd1a_hs_g --> cmp_g + gt1a_hs_g
could not calculate fold changes for ST8SIA54g: cmpacna_g + gm1b_hs_g --> cmp_g + gd1c_hs_g + 2.0 h_g
could not calculate fold changes for ST8SIA54g: cmpacna_g + gm1b_hs_g --> cmp_g + gd1c_hs_g + 2.0 h_g
could not calculate fold changes for

could not calculate fold changes for ST8SIA55g: cmpacna_g + gq1c_hs_g --> cmp_g + gp1calpha_hs_g + h_g
could not calculate fold changes for T2M26DCOAHLx: h2o_x + t2m26dcoa_x <=> 3h26dm5coa_x
could not calculate fold changes for STRDNCCPT1: crn_c + strdnccoa_c --> coa_c + strdnccrn_c
could not calculate fold changes for ST8SIA56g: acngal14acglcgalgluside_hs_g + cmpacna_g --> acnacngal14acglcgalgluside_hs_g + cmp_g + 2.0 h_g
could not calculate fold changes for SUCRe: h2o_e + sucr_e --> fru_e + glc_D_e
could not calculate fold changes for STS1: dheas_c + h2o_c --> dhea_c + h_c + so4_c
could not calculate fold changes for TETPENT3CPT1: crn_c + tetpent3coa_c --> coa_c + tetpent3crn_c
could not calculate fold changes for STRDNCCPT1: crn_c + strdnccoa_c --> coa_c + strdnccrn_c
could not calculate fold changes for T4HCINNOX: 2.0 T4hcinnm_c + o2_c --> 2.0 34dhcinm_c
could not calculate fold changes for TETPENT3CPT1: crn_c + tetpent3coa_c --> coa_c + tetpent3crn_c
could not calculate fold chang